In [0]:
events = spark.table("ecommerce.silver.silver_events")

In [0]:
from pyspark.sql.functions import hour, dayofweek

events_ml = events \
    .withColumn("hour", hour("event_time")) \
    .withColumn("day_of_week", dayofweek("event_time")) \
    .filter(events.price.isNotNull())

In [0]:
train_df, test_df = events_ml.randomSplit([0.5, 0.5], seed=42)

In [0]:
print("Rows:", events.count())
print("Train:", train_df.count())
print("Test:", test_df.count())

Rows: 109819992
Train: 54914763
Test: 54905229


In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [0]:
indexer = StringIndexer(
    inputCol="event_type",
    outputCol="event_type_index"
)
assembler = VectorAssembler(
    inputCols=["event_type_index", "hour", "day_of_week"],
    outputCol="features"
)

In [0]:
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, RandomForestRegressor

In [0]:
lr = LinearRegression(labelCol="price")

dt = DecisionTreeRegressor(
    labelCol="price",
    featuresCol="features",
    maxDepth=3   
)
rf = RandomForestRegressor(
    labelCol="price",
    featuresCol="features",
    numTrees=10,     
    maxDepth=5,     
    seed=42
)

In [0]:
lr_pipeline = Pipeline(stages=[indexer, assembler, lr])
dt_pipeline = Pipeline(stages=[indexer, assembler, dt])
rf_pipeline = Pipeline(stages=[indexer, assembler, rf])

In [0]:
import mlflow
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
evaluator = RegressionEvaluator(
    labelCol="price",
    predictionCol="prediction",
    metricName="r2"
)

In [0]:
models = {
    "Linear Regression": lr_pipeline,
    "Decision Tree": dt_pipeline,
    "Random Forest": rf_pipeline
}

In [0]:
for name, pipeline in models.items():
    with mlflow.start_run(run_name=name):
        model = pipeline.fit(train_df)
        preds = model.transform(test_df)
        r2 = evaluator.evaluate(preds)

        mlflow.log_metric("r2_score", r2)
        mlflow.spark.log_model(model, "model", dfs_tmpdir="/Volumes/workspace/default/ml_flow_models")

        print(f"{name} R²: {r2:.3f}")

2026/01/21 09:52:10 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/21 09:52:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-5fdd8b21-1219-4cf1-b7b2-e0/tmp_98a4kp7/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/21 09:52:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Linear Regression R²: 0.000


2026/01/21 09:53:58 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/21 09:54:01 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-5fdd8b21-1219-4cf1-b7b2-e0/tmpe1ts1wld/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/21 09:54:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Decision Tree R²: 0.001


2026/01/21 09:56:57 WARNING mlflow.utils.requirements_utils: Found pyspark version (4.0.0+databricks.connect.17.2.2) contains a local version label (+databricks.connect.17.2.2). MLflow logged a pip requirement for this package as 'pyspark==4.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2026/01/21 09:57:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-5fdd8b21-1219-4cf1-b7b2-e0/tmpw_ifcea4/model, flavor: spark). Fall back to return ['pyspark==4.0.0']. Set logging level to DEBUG to see the full traceback. 
2026/01/21 09:57:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Random Forest R²: 0.001
